In [1]:
import pandas as pd
import numpy as np

In [2]:
financial_metrics_path = 'financialMetrics.csv'
piotroski_scores_path = 'piotroskiScores.csv'
valuation_metrics_path = 'valuationMetrics.csv'
companies_path = 'companies.csv'

financial_metrics_df = pd.read_csv(financial_metrics_path)
piotroski_scores_df = pd.read_csv(piotroski_scores_path)
valuation_metrics_df = pd.read_csv(valuation_metrics_path)
companies_df = pd.read_csv(companies_path)


In [7]:
print(companies_df)

     Unnamed: 0 Symbol              Security             GICS Sector  \
0             0    MMM                    3M             Industrials   
1             1    AOS           A. O. Smith             Industrials   
2             2    ABT                Abbott             Health Care   
3             3   ABBV                AbbVie             Health Care   
4             4    ACN             Accenture  Information Technology   
..          ...    ...                   ...                     ...   
498         498    YUM           Yum! Brands  Consumer Discretionary   
499         499   ZBRA    Zebra Technologies  Information Technology   
500         500    ZBH         Zimmer Biomet             Health Care   
501         501   ZION  Zions Bancorporation              Financials   
502         502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, 

In [5]:
companies_df

Index(['Unnamed: 0', 'Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

In [26]:

class StockAnalysis:
    def __init__(self, financial_metrics_path = 'financialMetrics.csv', piotroski_scores_path = 'piotroskiScores.csv', valuation_metrics_path = 'valuationMetrics.csv', companies_path = 'companies.csv', summary_details_path = 'summaryDetails.csv'):
        self.financial_metrics_path = financial_metrics_path
        self.piotroski_scores_path = piotroski_scores_path
        self.valuation_metrics_path = valuation_metrics_path
        self.companies_path = companies_path
        self.summary_details_path = summary_details_path

    def load_data(self):
        self.financial_metrics_df = pd.read_csv(self.financial_metrics_path)
        self.piotroski_scores_df = pd.read_csv(self.piotroski_scores_path)
        self.valuation_metrics_df = pd.read_csv(self.valuation_metrics_path)
        self.companies_df = pd.read_csv(self.companies_path)
        self.summary_details_df = pd.read_csv(self.summary_details_path)

    def preprocess_data(self):
        # Preprocess Valuation Metrics for latest date
        self.latest_valuation_metrics_df = self.valuation_metrics_df.sort_values(by='asOfDate', ascending=False).drop_duplicates('Ticker')
        
        # Sort by Ticker and Date, then drop duplicates, keeping the last (most recent)
        self.piotroski_scores_df.sort_values(by=['Ticker', 'Date'], ascending=[True, False], inplace=True)
        self.piotroski_scores_df.drop_duplicates(subset='Ticker', keep='first', inplace=True)
        
        # Rename 'Symbol' to 'Ticker' in companies_df
        self.companies_df.rename(columns={'Symbol': 'Ticker'}, inplace=True)

    def merge_data(self):
        # Merge all dataframes
        self.merged_df = pd.merge(pd.merge(pd.merge(self.financial_metrics_df, self.piotroski_scores_df, on='Ticker'), self.latest_valuation_metrics_df), self.summary_details_df, on='Ticker')
        self.merged_df = pd.merge(self.merged_df, self.companies_df[['Ticker', 'Security', 'GICS Sub-Industry']], on='Ticker')
        
    def calculate_percent_upside(self):
        # Ensure the necessary columns exist
        if 'currentPrice' in self.financial_metrics_df and 'targetMeanPrice' in self.financial_metrics_df:
            self.financial_metrics_df['Percent Upside'] = ((self.financial_metrics_df['targetMeanPrice'] - self.financial_metrics_df['currentPrice']) / self.financial_metrics_df['currentPrice']) * 100
        else:
            # If columns don't exist, fill with NaN or a default value
            self.financial_metrics_df['Percent Upside'] = np.nan

    def calculate_bull_bear_indicator(self):
        self.summary_details_df['Market Trend'] = np.where(self.summary_details_df['fiftyDayAverage'] > self.summary_details_df['twoHundredDayAverage'], 'Bullish', 'Bearish')

    def clean_outliers(self):
        # Calculate sector average EV/EBITDA
        sector_avg_ebitda_ratio = self.merged_df.groupby('GICS Sub-Industry')['EnterprisesValueEBITDARatio'].mean()
        self.merged_df['Sector Avg EV/EBITDA'] = self.merged_df['GICS Sub-Industry'].map(sector_avg_ebitda_ratio)

        # Filter out negative and extreme EV/EBITDA ratios
        self.merged_df = self.merged_df[self.merged_df.EnterprisesValueEBITDARatio > 0]
        self.merged_df['EV/EBITDA % Diff'] = ((self.merged_df['Sector Avg EV/EBITDA'] - self.merged_df['EnterprisesValueEBITDARatio']) / self.merged_df['EnterprisesValueEBITDARatio']) * 100
        self.merged_df = self.merged_df[abs(self.merged_df['EV/EBITDA % Diff']) <= 100]
        
        self.financial_metrics_df = self.financial_metrics_df[abs(self.financial_metrics_df['Percent Upside']) <= 100]

    def calculate_composite_score(self):
        # Normalize metrics and calculate composite score
        self.merged_df['Piotroski Score Norm'] = self.merged_df['Piotroski Score'] / 9
        self.merged_df['Percent Upside Norm'] = (self.merged_df['Percent Upside'] - self.merged_df['Percent Upside'].min()) / (self.merged_df['Percent Upside'].max() - self.merged_df['Percent Upside'].min())
        self.merged_df['EV/EBITDA % Diff Norm'] = (self.merged_df['EV/EBITDA % Diff'] - self.merged_df['EV/EBITDA % Diff'].min()) / (self.merged_df['EV/EBITDA % Diff'].max() - self.merged_df['EV/EBITDA % Diff'].min())
        self.merged_df['Beta Norm'] = (self.merged_df['beta'] - self.merged_df['beta'].min()) / (self.merged_df['beta'].max() - self.merged_df['beta'].min())
        self.merged_df['Composite Score'] = np.mean(self.merged_df[['Piotroski Score Norm', 'Percent Upside Norm', 'EV/EBITDA % Diff Norm', 'Beta Norm']], axis=1)

    def get_top_stocks_per_sector(self):
        # Select top 5 stocks per sector based on the composite score
        self.top_stocks_by_sector = self.merged_df.groupby('GICS Sub-Industry').apply(lambda x: x.nlargest(3, 'Composite Score')).reset_index(drop=True)
        return self.top_stocks_by_sector[['Ticker', 'Security', 'GICS Sub-Industry', 'Piotroski Score', 'Piotroski Score Norm', 'Percent Upside', 'Percent Upside Norm', 'EnterprisesValueEBITDARatio', 'Sector Avg EV/EBITDA', 'EV/EBITDA % Diff', 'EV/EBITDA % Diff Norm', 'beta', 'Beta Norm', 'Market Trend', 'Composite Score']]

    def get_top_10_stocks(self):
        # Sort the merged data by composite score and select the top 10
        top_10_stocks = self.merged_df.sort_values(by='Composite Score', ascending=False).head(10)
        return top_10_stocks[['Ticker', 'Security', 'GICS Sub-Industry', 'Piotroski Score', 'Piotroski Score Norm', 'Percent Upside', 'Percent Upside Norm', 'EnterprisesValueEBITDARatio', 'Sector Avg EV/EBITDA', 'EV/EBITDA % Diff', 'EV/EBITDA % Diff Norm', 'beta', 'Beta Norm', 'Market Trend', 'Composite Score']]

stock_analysis = StockAnalysis()
stock_analysis.load_data()
stock_analysis.calculate_percent_upside()
stock_analysis.calculate_bull_bear_indicator()
stock_analysis.preprocess_data()
stock_analysis.merge_data()
stock_analysis.clean_outliers()
stock_analysis.calculate_composite_score()

In [27]:
stock_analysis.get_top_stocks_per_sector()

,Ticker,Security,GICS Sub-Industry,Piotroski Score,Piotroski Score Norm,Percent Upside,Percent Upside Norm,EnterprisesValueEBITDARatio,Sector Avg EV/EBITDA,EV/EBITDA % Diff,EV/EBITDA % Diff Norm,beta,Beta Norm,Market Trend,Composite Score
0,OMC,Omnicom Group,Advertising,8,0.888889,10.499939,0.337738,29.263702,29.773631,1.742531,0.490161,0.947,0.343838,Bearish,0.515157
1,IPG,Interpublic Group of Companies (The),Advertising,6,0.666667,7.729160,0.309160,30.283560,29.773631,-1.683848,0.471549,1.166,0.432323,Bearish,0.469925
2,DE,John Deere,Agricultural & Farm Machinery,3,0.333333,5.536726,0.286547,36.937864,36.937864,0.000000,0.480696,1.081,0.397980,Bearish,0.374639
3,ADM,ADM,Agricultural Products & Services,6,0.666667,22.842302,0.465040,33.746915,30.801000,-8.729435,0.433278,0.836,0.298990,Bearish,0.465994
4,BG,Bunge Global SA,Agricultural Products & Services,5,0.555556,23.312883,0.469893,27.855085,30.801000,10.575861,0.538143,0.654,0.225455,Bullish,0.447262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,GPN,Global Payments,Transaction & Payment Processing Services,8,0.888889,18.741130,0.422740,43.381399,56.806894,30.947583,0.648799,0.977,0.355960,Bullish,0.579097
231,PYPL,PayPal,Transaction & Payment Processing Services,6,0.666667,15.808885,0.392496,40.606932,56.806894,39.894571,0.697398,1.376,0.517172,Bearish,0.568433
232,FI,Fiserv,Transaction & Payment Processing Services,7,0.777778,9.065156,0.322940,41.011220,56.806894,38.515493,0.689907,0.893,0.322020,Bearish,0.528161
233,AWK,American Water Works,Water Utilities,8,0.888889,10.777544,0.340602,50.738684,50.738684,0.000000,0.480696,0.626,0.214141,Bearish,0.481082


In [28]:
stock_analysis.get_top_10_stocks()

,Ticker,Security,GICS Sub-Industry,Piotroski Score,Piotroski Score Norm,Percent Upside,Percent Upside Norm,EnterprisesValueEBITDARatio,Sector Avg EV/EBITDA,EV/EBITDA % Diff,EV/EBITDA % Diff Norm,beta,Beta Norm,Market Trend,Composite Score
73,CCL,Carnival,"Hotels, Resorts & Cruise Lines",8,0.888889,6.598985,0.297503,23.225094,38.722618,66.727497,0.843152,2.455,0.953131,Bearish,0.745669
189,HAL,Halliburton,Oil & Gas Equipment & Services,8,0.888889,29.461681,0.533313,34.278359,38.414410,12.066071,0.546237,2.140,0.825859,Bullish,0.698575
167,FSLR,First Solar,Semiconductors,6,0.666667,43.284048,0.675880,42.647039,70.546170,65.418683,0.836042,1.417,0.533737,Bearish,0.678082
344,RCL,Royal Caribbean Group,"Hotels, Resorts & Cruise Lines",6,0.666667,9.392215,0.326313,25.568791,38.722618,51.444853,0.760138,2.460,0.955152,Bullish,0.677067
243,LVS,Las Vegas Sands,Casinos & Gaming,8,0.888889,42.286325,0.665589,40.213154,55.662103,38.417649,0.689376,1.199,0.445657,Bearish,0.672378
295,NXPI,NXP Semiconductors,Semiconductors,9,1.000000,7.798522,0.309875,45.301619,70.546170,55.725495,0.783390,1.570,0.595556,Bullish,0.672205
290,NCLH,Norwegian Cruise Line Holdings,"Hotels, Resorts & Cruise Lines",6,0.666667,0.669915,0.236350,26.142244,38.722618,48.122777,0.742093,2.571,1.000000,Bearish,0.661277
38,APTV,Aptiv,Automotive Parts & Equipment,7,0.777778,36.686109,0.607827,48.114106,43.639332,-9.300338,0.430177,2.065,0.795556,Bearish,0.652835
401,VTRS,Viatris,Pharmaceuticals,9,1.000000,50.586980,0.751204,54.951610,48.546282,-11.656306,0.417380,1.165,0.431919,Bearish,0.650126
222,IQV,IQVIA,Life Sciences Tools & Services,7,0.777778,3.900235,0.269668,58.713789,112.145773,91.004149,0.975020,1.418,0.534141,Bearish,0.639152
